# exploration 13번째 과제
@ 황한용(3기/쏘카)

## 라이브러리 선언

In [1]:
import implicit
from implicit.als import AlternatingLeastSquares
import numpy as np
import os
import pandas as pd
import scipy
from scipy.sparse import csr_matrix

# implicit 라이브러리에서 권장하고 있는 부분입니다.
# 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

## 상수선언

In [2]:
BASE_PATH = os.getenv('HOME') + "/aiffel/recommendata_iu/data/ml-1m" # 기본 데이터경로
MF_P_DATA_PATH = BASE_PATH + "/ratings.dat" # 영화에 관한 사용자의 평가가 들어가있는 데이터경로
MF_Q_DATA_PATH = BASE_PATH + "/movies.dat"# 영화에 관한 특성데이터가 들어가있는 데이터경로
MF_P_COLS = ['user_id', 'movie_id', 'counts', 'timestamp'] # 영화에 관한 사용자의 특성(MF model P feature)
MF_Q_COLS = ['movie_id', 'title', 'genre']  # 영화에 관한 특성데이터(MF Q model feature)
READ_CSV_KWARGS = dict(
    sep='::'
    , encoding='ISO-8859-1'
    , engine='python'
)
MY_NICKNAME = "hhyong"
MY_FAVORIT_LIST = ['명량 (2014)' , '극한직업 (2019)' ,'7번방의 선물 (2013)' ,'암살 (2015)' ,'범죄도시 (2017)']
MY_FAVORIT_LIST_TOP = ["Good Will Hunting (1997)"   
                        ,"Cold Comfort Farm (1995)" 
                        , "Chinatown (1974)"
                        ,"Pinocchio (1940)"
                        ,"Godfather: Part II, The (1974)"]

## 메인

### 전처리

In [3]:
ratings = pd.read_csv(
        MF_P_DATA_PATH
        , names=MF_P_COLS
        , **READ_CSV_KWARGS
)
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


indexing까지 완료된 사용자, 영화,평점 데이터를 로드한다.<br>
MF model에 사용되는 데이터 중 K vector데이터이다.

In [4]:
orginal_data_size = len(ratings)
ratings = ratings[ratings['counts'] > 2]
filtered_data_size = len(ratings)

print(
    f"""orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}
Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}"""
)

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


시청횟수 3점 이하의 데이터는 모두 drop한다.<br>
그 후 로드시의 데이터와 filtering한 데이터의 수, filtering한 데이터의 %를 확인한다.<br>
한사람당 시청횟수가 3번미만인 데이터는 83.63%로 시청횟수 데이터의 16.37%가 제거된다.

In [5]:
movies = pd.read_csv(
    MF_Q_DATA_PATH
    , names=MF_Q_COLS
    , **READ_CSV_KWARGS
)
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


영화의 속성(장르)에 관한 정보를 가져온다.<br>
`title`은 제목과 연도가 합쳐저있으며 <br>
`genre`의 정보는 `|`를 구분자로 여러개 정의되어있다.

In [6]:
num_user_rating = ratings['user_id'].nunique()
num_movie_rating = ratings['movie_id'].nunique()
num_movie_movies = movies['movie_id'].nunique()

print("total user_id in rating: ", num_user_rating)
print("total movie_id in rating: ", num_movie_rating)
print("total movie_id in movies: ", num_movie_movies)

total user_id in rating:  6039
total movie_id in rating:  3628
total movie_id in movies:  3883


유저가 본 영화와 `movies`에 들어있는 데이터의 영화의 갯수를 보면 `rating`의 데이터가 조금 작다.<br>
그러므로 유저가 본 데이터의 영화를 기준으로 merge를 해야 유저가 평가한 영화의 순위등을 볼 수 있다.<br>

In [7]:
movie_info = pd.merge(ratings, movies, on="movie_id", how="left")
ori_len = len(movie_info)
movie_info.dropna(inplace=True)
dropna_len = len(movie_info)

print("original rating len: ", ori_len)
print("dropped rating len: ", dropna_len)

print("top 30 movie")
print(
    movie_info.set_index("movie_id").iloc[
        movie_info.groupby(["movie_id"]).sum().sort_values(by=["counts"], ascending=False).index[:30].to_list()
    ][["title", "genre"]]
)

original rating len:  836478
dropped rating len:  836478
top 30 movie
                                           title  \
movie_id                                           
1704                    Good Will Hunting (1997)   
728                     Cold Comfort Farm (1995)   
1252                            Chinatown (1974)   
596                             Pinocchio (1940)   
1221              Godfather: Part II, The (1974)   
1193      One Flew Over the Cuckoo's Nest (1975)   
24                                 Powder (1995)   
1544       Lost World: Jurassic Park, The (1997)   
2858                      American Beauty (1999)   
2396                  Shakespeare in Love (1998)   
36                       Dead Man Walking (1995)   
527                      Schindler's List (1993)   
1442                          Prefontaine (1997)   
1509                          All Over Me (1997)   
2916                         Total Recall (1990)   
2174                          Beetlejuice (198

유저가 많아 본 영화를 각 유저의 시청횟수(`counts`)합산 기준으로 출력하였다.<br>
유저가 본 영화 중, 영화 메타데이터에 없는 영화는 없다. 

In [8]:
movie_info["title"].nunique() == movie_info["movie_id"].nunique()

True

영화의 타이틀 중 중복되지 않는 갯수와 `movie_id`의 갯수는 동일하므로 `title`은 `movie_id`을 대표한다고 볼 수 있다.<br>
또한 `genre`, `timestamp`는 이번 추천시스템에서 사용하지 않은 계획이므로 없어도 무방하다.<br>
먼저 아래와 같이 필요없는 데이터는 `drop()`을 이용해 제거한다.<br>

In [9]:
movie_info.drop(["movie_id", "timestamp", "genre"], axis=1, inplace=True)

In [10]:
movie_info

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,1,3,James and the Giant Peach (1996)
2,1,3,My Fair Lady (1964)
3,1,4,Erin Brockovich (2000)
4,1,5,"Bug's Life, A (1998)"
...,...,...,...
836473,6040,3,Platoon (1986)
836474,6040,5,"Crying Game, The (1992)"
836475,6040,5,Welcome to the Dollhouse (1995)
836476,6040,4,Sophie's Choice (1982)


데이터를 확인했을 시, 필요한 데이터만 제거된 모습을 확인할 수 있다.<br>

In [11]:
max_counts = ratings["counts"].max()

my_playlist = pd.DataFrame({'user_id': [MY_NICKNAME]*5, 'title': MY_FAVORIT_LIST, 'counts':[max_counts]*5})

if not movie_info.isin({'user_id':[MY_NICKNAME]})['user_id'].any():
    movie_info = movie_info.append(my_playlist)

movie_info.reset_index(drop=True, inplace=True)
movie_info.tail(10)

,user_id,counts,title
836473,6040,3,Platoon (1986)
836474,6040,5,"Crying Game, The (1992)"
836475,6040,5,Welcome to the Dollhouse (1995)
836476,6040,4,Sophie's Choice (1982)
836477,6040,4,E.T. the Extra-Terrestrial (1982)
836478,hhyong,5,명량 (2014)
836479,hhyong,5,극한직업 (2019)
836480,hhyong,5,7번방의 선물 (2013)
836481,hhyong,5,암살 (2015)
836482,hhyong,5,범죄도시 (2017)


줄수 있는 최대한의 평점을 구한후, <br>
좋아하는 영화 5개를 넣고 임의로 최고평점을 넣었다.<br>

In [12]:
csr_matrix((movie_info["counts"], (movie_info['user_id'], movie_info['title'])), shape= (movie_info['user_id'].nunique(), movie_info['title'].nunique()))

ValueError: invalid literal for int() with base 10: 'hhyong'

유저와 영화의 유니크한 백터를 구하며 이를 사용해 `csr_matrix`를 생성한다.<br>
`indexing`이 되어있지 않으므로 에러가 나타난다.

In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = movie_info['user_id'].unique()
movie_unique = movie_info['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[MY_NICKNAME])
print(movie_to_idx[MY_FAVORIT_LIST[0]])

# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_info = movie_info['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_info) == len(movie_info):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_info['user_id'] = temp_user_info   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_info = movie_info['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_info) == len(movie_info):
    print('title column indexing OK!!')
    movie_info['title'] = temp_movie_info
else:
    print('title column indexing Fail!!')

movie_info

6039
3628
user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
836478,6039,5,3628
836479,6039,5,3629
836480,6039,5,3630
836481,6039,5,3631


`title`, `user_id`모두 유니크한 값이므로 indexing을 통해 새로들어온 값을 보간한다.<br>
`user_id`, `movie_id` 두 컬럼 다 `NaN`값이 유니크한 정수형으로 대체된 모습을 볼 수 있다.

In [14]:
csr_matrix_data = csr_matrix(
    (movie_info["counts"], (movie_info['user_id'], movie_info['title']))
    , shape= (movie_info['user_id'].nunique(), movie_info['title'].nunique())
)
csr_matrix_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

indexing으로 보간한 값을 사용했을 시, CSR Matrix가 정상적으로 생성된 것을 볼 수 있다.<br>

In [15]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(
    factors=100
    , regularization=0.001
    , use_gpu=False
    , iterations=14
    , dtype=np.float32
)

아래의 내용을 보고 참고하여 파라미터를 변경시켰다.
- factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미입니다.

https://yeomko.tistory.com/4 의 마지막 내용을 보면 논문에서 `iterations`은 10~15사이의 값을 사용한다고 하여 사이의 값을 넣었다.

In [16]:
als_model.fit(csr_matrix_data.T)

  0%|          | 0/14 [00:00<?, ?it/s]

학습시에는 CSR Matrix의 전치된 값을 사용하므로 전치된 행렬을 학습에 넣었다.

In [17]:
for i in range(len(MY_FAVORIT_LIST)):
    user_hhyong, favorit1 = user_to_idx[MY_NICKNAME], movie_to_idx[MY_FAVORIT_LIST[i]]
    hhyong_vector, favorit1_vector = als_model.user_factors[user_hhyong], als_model.item_factors[favorit1]
    print(np.dot(hhyong_vector, favorit1_vector))

0.0036423083
0.0040582754
0.0041720644
0.004047983
0.0036109076


전혀 없는 5개의 영화 중 하나를 선택하여 선호도를 파악하였을 경우 소수점 4자리까지 내려간 모습을 볼 수 있다.<br>
그렇다면 선호도 TOP 5위 안의 데이터를 이용하여 선호도를 선호하였을경우를 살펴보겠다.

In [18]:
my_playlist = pd.DataFrame({'user_id': [user_to_idx[MY_NICKNAME]]*5, 'title': [movie_to_idx[m] for m in MY_FAVORIT_LIST_TOP], 'counts':[max_counts]*5})
movie_info = movie_info.append(my_playlist)
movie_info

,user_id,counts,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,5,248
1,6039,5,225
2,6039,5,500
3,6039,5,528


`counts`칼럼기준 top5 영화를 선택하여 평점기록에 넣었다.

In [19]:
als_model.fit(csr_matrix(
    (movie_info["counts"], (movie_info['user_id'], movie_info['title']))
    , shape= (movie_info['user_id'].nunique(), movie_info['title'].nunique())
).T)

  0%|          | 0/14 [00:00<?, ?it/s]

In [20]:
for i in range(len(MY_FAVORIT_LIST_TOP)):
    user_hhyong, favorit1 = user_to_idx[MY_NICKNAME], movie_to_idx[MY_FAVORIT_LIST_TOP[i]]
    hhyong_vector, favorit1_vector = als_model.user_factors[user_hhyong], als_model.item_factors[favorit1]
    print(np.dot(hhyong_vector, favorit1_vector))

0.3325468
0.13762872
0.42816117
0.24852826
0.66537744


임시로 csr_matrix를 구현하여 데이터를 집어넣었을 시<br>
선호도를 구하였을 시 신규 영화보다 월등히 선호도가 올라간 모습을 볼 수 있다.<br>
다른 영화를 구햐였을시 최대 65퍼센트 이상인 영화도 있으며 적어도 14퍼센트 이상인 점을 확인가능하다.<br>
이는 협업필터링 방식의 단점을 확실히 보여준다.

In [21]:
favorite_movie = '범죄도시 (2017)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3632, 1.0000001),
 (3628, 0.9257034),
 (3631, 0.8808701),
 (3630, 0.8792641),
 (3629, 0.8640779),
 (3517, 0.76700985),
 (3121, 0.74131244),
 (2435, 0.7401872),
 (2752, 0.7363719),
 (3434, 0.7357467),
 (3015, 0.73529375),
 (3430, 0.7341199),
 (2808, 0.7340552),
 (3564, 0.7338295),
 (3367, 0.7313926)]

In [22]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['범죄도시 (2017)',
 '명량 (2014)',
 '암살 (2015)',
 '7번방의 선물 (2013)',
 '극한직업 (2019)',
 'Paralyzing Fear: The Story of Polio in America, A (1998)',
 'Touch (1997)',
 'Wide Awake (1998)',
 'Lady of Burlesque (1943)',
 'Prom Night IV: Deliver Us From Evil (1992)',
 'H.O.T.S. (1979)',
 'Gumby: The Movie (1995)',
 'Carnosaur 2 (1995)',
 'Impact (1949)',
 'Wife, The (1995)']

`범죄도시` 영화같은 신규영화를 검색시 전혀 비슷하지 않은 영화가 나온다.

In [23]:
favorite_movie = 'Pi (1998)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Pi (1998)',
 'Gattaca (1997)',
 'City of Lost Children, The (1995)',
 'Strange Days (1995)',
 'Dark City (1998)',
 'Cube (1997)',
 'Delicatessen (1991)',
 'eXistenZ (1999)',
 'Twelve Monkeys (1995)',
 'Until the End of the World (Bis ans Ende der Welt) (1991)',
 'Lost Highway (1997)',
 'Contact (1997)',
 'X-Files: Fight the Future, The (1998)',
 'Fifth Element, The (1997)',
 'Fear and Loathing in Las Vegas (1998)']

`PI`영화같은 디스토피아, 스릴러,  공상영화에 가까운 영화를 추천했을 시 비슷한 영화가 나온다.<br>
적중률이 대단히 높다.

In [24]:
user = user_to_idx[MY_NICKNAME]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_matrix_data, N=20, filter_already_liked_items=True)
movie_recommended

[(607, 0.70051974),
 (380, 0.6653775),
 (500, 0.42816114),
 (248, 0.33254683),
 (224, 0.2862421),
 (157, 0.25257936),
 (528, 0.24852827),
 (1096, 0.23839429),
 (269, 0.23216668),
 (128, 0.23121926),
 (17, 0.22467238),
 (121, 0.22068575),
 (435, 0.21864986),
 (601, 0.2144154),
 (37, 0.19951695),
 (46, 0.19171537),
 (572, 0.1891605),
 (323, 0.18890789),
 (291, 0.18862768),
 (385, 0.1851765)]

유저에 대한 추천영화를 20개 추천했을시의 백터이다.

In [25]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Godfather, The (1972)',
 'Godfather: Part II, The (1974)',
 'Chinatown (1974)',
 'Good Will Hunting (1997)',
 'L.A. Confidential (1997)',
 'Shawshank Redemption, The (1994)',
 'Pinocchio (1940)',
 'Manchurian Candidate, The (1962)',
 'GoodFellas (1990)',
 'French Connection, The (1971)',
 'Bambi (1942)',
 'Silence of the Lambs, The (1991)',
 'Godfather: Part III, The (1990)',
 'Citizen Kane (1941)',
 'Cinderella (1950)',
 'Dumbo (1941)',
 'Sleeping Beauty (1959)',
 'Dead Man Walking (1995)',
 'Maltese Falcon, The (1941)',
 'Truman Show, The (1998)']

해당 백터를 보았을 시 새로넣은 데이터에 관해서는 추천이 전혀 없으며,<br>
나중에 넣은 상위 5개에 관한 영화가 대다수를 차지하는 것을 볼 수 있다.

In [26]:
gp = movie_to_idx['Godfather, The (1972)']
explain = als_model.explain(user, csr_matrix_data, itemid=gp)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('7번방의 선물 (2013)', 0.006447665278136984),
 ('범죄도시 (2017)', 0.006436285388732623),
 ('명량 (2014)', 0.006362890789819597),
 ('암살 (2015)', 0.006303574838352918),
 ('극한직업 (2019)', 0.0061766046445112235)]

상위 영화에 대한 기여도를 보았을 시, 대단히 적은 값이 나온다.<br>
이는 데이터가 신규이기 때문에 생기는 문제로 위에서도 언급했다.

In [27]:
gp = movie_to_idx['7번방의 선물 (2013)']
explain = als_model.explain(user, csr_matrix_data, itemid=gp)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('극한직업 (2019)', 0.00017927431970396268),
 ('7번방의 선물 (2013)', 0.00017159275654632844),
 ('암살 (2015)', 0.00016953739970943096),
 ('명량 (2014)', 0.00014878433299700413),
 ('범죄도시 (2017)', 0.00012204185852085088)]

신규또한 마찬가지이다.

### 회고


- 협업필터링의 단점에 대해 확실히 알게되었다.
- 데이터의 질도 중요하지만 얼마나 많은 데이터를 수집하는가도 추천시스템에 영향이 큰것을 알게되었다.